In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

**Задача интерполяции**: по данному набору значений функции $u(x)$ на сетке ${x_i}$ восстановить функцию $U(x)$, совпадающую с $u(x)$ в узлах $x_i$.

**Интерполяция кубическими сплайнами**: на каждом отрезке $[x_{i-1}, x_i]$ искомая функция - многочлен 3-й степени $S_i(x)$. $S_i(x)$ представлен в виде:

$S_{3,i}(x) = a_{0,i} + a_{1,i}(x-x_i) + a_{2,i}(x-x_i)^2 + a_{3,i}(x-x_i)^3$

Тогда задача интерполяции сводится к нахождению совокупности этих многочленов, при условии, что они достаточно гладкие (дважды дифференцируемые):

$S_3(x) = \bigcup_{i=0}^{n-1}S_{3,i}(x) \in C_2[a, b]$









### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

Функциональные:

(1) $\delta S_{3,i}(x)|_{x=x_i} = S_{3,i}(x_i) - f(x_i) = 0,$

(2) $\delta S_{3,i}(x)|_{x=x_i+1} = S_{3,i}(x_{i+1}) - f(x_{i+1}) = 0;$

Дифференциальные:

(3) $\delta S''_{3,i}(x)|_{x=x_i} = S''_{3,i}(x_i) - f''(x_i) = 0,$

(4) $\delta S''_{3,i}(x)|_{x=x_i+1} = S''_{3,i}(x_{i+1}) - f''(x_{i+1}) = 0.$

### 3. Formulate stitching conditions [mathematical formula]

$S^{p}_{3, i-1}(x)|_{x=x_i} = S^{p}_{3, i}(x)|_{x=x_i}$

$i = \overline{1, n-1}$

$0 \leq p \leq 3, p \in \mathbb{Z}$

### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

Условия стыковки вместе с дифференциальными условиями согласования обеспечивают непрерывность $S^{p}_3(x), p=0,1,2$, во всех внутренних узлах $x_i$, определяющих точки стыковки звеньев, т.е. $(x_i, S_3(x_i)), (i=\overline{1,n-1})$. Это гарантирует выполнение условия $S_m(x) \in C_2[a,b]$.

### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

$S_{3,i}(x) = a_{0,i} + a_{1,i}(x-x_i) + a_{2,i}(x-x_i)^2 + a_{3,i}(x-x_i)^3$

$S''_{3,i}(x) = 2a_{2,i} + 6a_{3,i}x - 6a_{3,i}x_i$

$h_{i+1} = x_{i+1} - x_i, \Delta f_i = f_{i+1} - f_i, m_i = f''(x_i), \Delta m_i = m_{i+1} - m_i$

**Из уравнения (1) п. 2:**

$a_{0,i} = f_i$

**Из уравнения (3) п. 2:**

$2a_{2,i} + 6a_{3,i}x_i - 6a_{3,i}x_i = f''(x_i)$

$2a_{2,i} = m_i$

$a_{2,i} = \frac{m_i}{2}$

**Из уравнения (4) п. 2 c подстановкой $a_{2,i}$:**

$m_i + 6a_{3,i}x_{i+1} - 6a_{3,i}x_i = f''(x_{i+1})$

$m_i + 6a_{3,i}h_{i+1} = m_{i+1}$

$a_{3,i} = \frac{\Delta m_i}{6h_{i+1}}$

**Из уравнения (2) п. 2 c подстановкой $a_{0,i}, a_{2,i}, a_{3,i}$:**

$ f_i + a_{1,i}(x_{i+1}-x_i) + \frac{m_i}{2}(x_{i+1}-x_i)^2 + \frac{\Delta m_i}{6h_{i+1}}(x_{i+1}-x_i)^3 = f_{i+1}$

$ f_i + a_{1,i}h_{i+1} + \frac{m_i}{2}h_{i+1}^2 + \frac{\Delta m_i}{6h_{i+1}}h_{i+1}^3 = f_{i+1}$

$ a_{1,i}h_{i+1} = -\frac{m_i}{2}h_{i+1}^2 - \frac{\Delta m_i}{6h_{i+1}}h_{i+1}^3 + \Delta f_i$

$ a_{1,i} = \frac{1}{h_{i+1}}\Delta f_i -\frac{h_{i+1}}{2}m_i - \frac{h_{i+1}}{6}\Delta m_i$

**Таким образом общая формула одного звена сплайна:**

$S_{3,i}(x) = f_i + (\frac{1}{h_{i+1}}\Delta f_i -\frac{h_{i+1}}{2}m_i - \frac{h_{i+1}}{6}\Delta m_i)(x-x_i) + \frac{m_i}{2}(x-x_i)^2 + \frac{\Delta m_i}{6h_{i+1}}(x-x_i)^3$

**Далее:**
$S'_{3,i} = S'_{3,i-1}$

$ S'_{3,i}(x) = \frac{\Delta f_i}{h_{i+1}} - \frac{h_{i+1}}{2}m_i - \frac{h_{i+1}}{6}\Delta m_i + m_i(x - x_i) + \frac{\Delta m_i}{2h_{i+1}}(x - x_i)^2, x \in [x_i, x_{i+1}]$

$ S'_{3,i-1}(x) = \frac{\Delta f_{i-1}}{h_{i}} - \frac{h_{i}}{2}m_{i-1} - \frac{h_{i}}{6}\Delta m_{i-1} + m_{i-1}(x -x_{i-1}) + \frac{\Delta m_{i-1}}{2h_{i}}(x-x_{i-1})^2, x \in [x_{i-1}, x_{i}]$

Т.к. производные пересекаются только в одной точке, можно подставить $x=x_i$:

$S'_{3,i}(x_i) = S'_{3,i-1}(x_i)$

$\frac{\Delta f_i}{h_{i+1}} - \frac{h_{i+1}}{2}m_i - \frac{h_{i+1}}{6}\Delta m_i = \frac{\Delta f_{i-1}}{h_{i}} - \frac{h_{i}}{2}m_{i-1}- \frac{h_{i}}{6}\Delta m_{i-1}+ m_{i-1}h_i + \frac{h_i}{2}\Delta m_{i-1}$

$\frac{\Delta f_i}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_{i}} =  \frac{h_{i+1}}{2}m_i + \frac{h_{i+1}}{6}(m_{i+1}-m_i) - \frac{h_{i}}{2}m_{i-1}- \frac{h_{i}}{6}(m_{i}-m_{i-1})$

$\frac{\Delta f_i}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_{i}} =  \frac{h_{i+1} + h_i}{3}m_i + \frac{h_{i+1}}{6}m_{i+1} + \frac{h_{i}}{6}m_{i-1}, i = \overline{1, n-1}$

### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

Matrix representation:
$$
\begin{bmatrix}
    \frac{h_1+h_2}{3} & \frac{h_2}{6}     & 0                 & \dots & 0     & 0 
    & \frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1} \\
    \frac{h_2}{6}     & \frac{h_2+h_3}{3} & \frac{h_3}{6}     & \dots & 0     & 0   
    & \frac{\Delta f_2}{h_3} - \frac{\Delta f_1}{h_2}\\
    0                 & \frac{h_3}{6}     & \frac{h_3+h_4}{3} & \dots & 0     & 0    
    & \frac{\Delta f_3}{h_4} - \frac{\Delta f_2}{h_3} \\
    \dots             & \dots             & \dots             & \dots & \dots & \dots & \dots \\
    0                 & 0                 & 0                 & \dots & \frac{h_{n-1}}{6} & \frac{h_{n-1}+h_n}{3}
    & \frac{\Delta f_{n-1}}{h_n} - \frac{\Delta f_{n-2}}{h_{n-1}}
\end{bmatrix}$$

### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

Данная система относительно $m_i (i = \overline{0,n})$ является незамкнутой, т.к. не хватает двух уравнений для $m_0$ и $m_n$. Существуют различные способы для аппроксимации производных на концах, простейший: $m_0 = m_n = 0$ - условия натурального сплайна.

### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

We will demonstrate Gauss elimination on the first two rows of the matrix:

**Step 1**: divide 1st row by $\frac{h_1+h_2}{3}$

Result (1st row):

$
\begin{bmatrix}
    1 & \frac{h_2}{2(h_1+h_2)} & 0 & \dots & 0 & (\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1})\frac{3}{h_1+h_2}
\end{bmatrix}
$

**Step 2**: subtract 1st row multiplied by $\frac{h_2}{6}$ from 2nd row:

Result (1st and 2nd rows):

$
\begin{bmatrix}
    1 & \frac{h_2}{2(h_1+h_2)} & 0 & \dots & 0 & (\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1})\frac{3}{h_1+h_2}\\
    0 & \frac{-h_2^2 + 4(h_1+h_2)(h_2+h_3)}{12(h_1+h_2)} & \frac{h_3}{6} & \dots & 0 & \frac{2(\Delta f_2 h_2 - \Delta f_1 h_3)h_1(h_1+h_2) - (\Delta f_1 h_1 - \Delta f_0 h_2)h_2 h_3}{2 h_1 h_2 h_3 (h_1 + h_2)}
\end{bmatrix}
$

**Step 3**: divide 2nd row by $\frac{-h_2^2 + 4(h_1+h_2)(h_2+h_3)}{12(h_1+h_2)}$:

Result (1st and 2nd rows):

$
\begin{bmatrix}
    1 & \frac{h_2}{2(h_1+h_2)} & 0 & \dots & 0 & (\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1})\frac{3}{h_1+h_2}\\
    0 & 1 & \frac{2h_3(h_1+h_2)}{4(h_2+h_3)(h_1+h_2)-h_2} & \dots & 0 & \frac{6 h_2^2 h_3 \Delta f_0 + 12(h_1+h_2) \Delta f_2 + h_2 h_1 - 12(h_1+h_2)\Delta f_1 h_1 h_3 - h_1 h_2 h_3 6 \Delta f_1}{h_1 h_2 h_3 (4(h_2+h_3)(h_1 + h_2) - h_2^2)}
\end{bmatrix}
$

And after performing these operations on other rows, we get following structure of matrix:

$$
\begin{bmatrix}
    1     & -P_1  & 0     & 0     & \dots & Q_1   \\
    0     & 1     & -P_2  & 0     & \dots & Q_2   \\
    0     & 0     & 1     & -P_3  & \dots & Q_3   \\
    \dots & \dots & \dots & \dots & \dots & \dots \\
    0     & 0     & 0     & 1     & \dots & Q_n   \\
\end{bmatrix}$$

where $P_i$ and $Q_i$ are computed using formaulas from question 9.

### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

Forward pass:

$P_i = \frac{\gamma_i}{\beta_i - \alpha P_{i-1}}, P_1 = \frac{\gamma_1}{\beta_1}$

$Q_i = \frac{\alpha_i Q_{i-1} - \delta_i}{\beta_i - \alpha P_{i-1}}, Q_1 = -\frac{\delta_1}{\beta_1}$

$i = \overline{1, n-1}$

Backward pass:

$x_n = \frac{\alpha_n Q_{n-1} - \delta_n}{\beta_n - \alpha_n P_{n-1}} = Q_n$

$x_i = P_ix_{i+1} + Q_i, i = \overline{1, n-1}$

### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [ ]:
### YOUR CODE HERE


### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

Пусть функция $f$ имеет на отрезке $[a,b]$ непрерывную производную четвертого порядка и $M_4 = \max\limits_{[a,b]}|f^{(4)}(x)|$. Тогда для интерполяционного кубического сплайна $S_3(x)$, удовлетворяющего граничным условиям, справедлива следующая оценка погрешности:

$\max\limits_{[a,b]}|f(x)-S_3(x)| \leqslant CM_4h^4_{max}$

Где $h_{max} = \max\limits_{1 \leqslant i \leqslant n} h_i$ - максимальная из длин частичных отрезков.

Если на точках $x_i$ известны значения производной $y'_i = f'(x_i)$, тогда на каждом частичном отрезке $[x_{i-1}, x_i]$ сплайн однозначно определяется значениями $y_{i-1}, y_i, y'_{i-1}, y'_i$ (локальный сплайн). В этом случае константа $C = \frac{1}{384}$.

### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

Forward pass - $O(n)$, backward pass - $O(n) \rightarrow O(2n) = O(n)$

### Congrats!